# Datensortierung und Umformatierung
_Weitere Informationen sind im Code in den mit # markierten Zeilen zu finden_

## Die Umformatierung
Die Rohdaten werden von einer xlsx Datei zu einer csv Datei umformatiert

### Die Imports 

In [13]:
# IBM Cloud
from ibm_botocore.client import Config
import ibm_boto3

import csv
import time

from datetime import datetime

## Spezifikation für das Funktionieren in der Cloud
_Die folgenden Codefelder werden nur im bereich der Cloud gebraucht und entfallen bei benutzung auf einem lokalen System_

An den Stellen an welchen die Punkte '.........' sind werden eigentlich die erforderlichen Infos eingesetzt, welche aber aus sicherheitstechnischen Gründen von mir entfernt wurden 

In [14]:
# @hidden_cell
# The following code contains the credentials for a connection in your Project.
# You might want to remove those credentials before you share your notebook.
cosCreds = {
    'IBM_API_KEY_ID': '........',
    'IAM_SERVICE_ID': 'crn:v1:bluemix:public:iam-identity::a/a6c0651c3aab4e74a4d1e131c1858b21::serviceid:ServiceId-cf18b8e2-291a-4175-b02f-4fa012271ab7',
    'ENDPOINT': 'https://s3.private.eu-de.cloud-object-storage.appdomain.cloud',
    'IBM_AUTH_ENDPOINT': '........',
    'BUCKET': '........'
}

#### Aus der Cloud lesen

In [15]:
def read_file_cos(credentials,local_file_name, key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    data_cos = None 
    try:
        res=cos.download_file(Bucket=credentials['BUCKET'],Key=key,Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Downloaded: %s' %(key))

#### In die Cloud lesen

In [16]:
def upload_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.upload_file(Filename=local_file_name, Bucket=credentials['BUCKET'],Key=key)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Uploaded: %s' %(key))

## Der eigentliche Code

In [26]:
start = time.time()
i=0
o=0



# resultfile
year = range(1987,2019,1)
for y in year:
    
    header = []
    
    # filenamen
    filenameBeg = "Halbstd-Werte-Stuttgart-Mitte-SZ_%d.csv" %(y)
    filenameEnd = "Halbstd-Werte-Stuttgart-Mitte-SZ_%d_D.csv" %(y)
    
    localFileBeg = "/var/tmp/%s" %(filenameBeg)
    localFileEnd = "/var/tmp/%s" %(filenameEnd)

    read_file_cos(cosCreds,localFileBeg,filenameBeg)
    
    # csv Endfile öffnen 
    with open(localFileEnd,'w', newline='') as csvfile:
        print(y)
        writer_csv = csv.writer(csvfile, delimiter=';')
        # lesefile
        for datafile in [localFileBeg]:
            with open(datafile,newline = '') as f:
                readers = csv.reader(f, delimiter= ',')
                for row in readers:
                    i += 1
                    
                    # 1. Spalte entfernen
                    row.pop(0)
                    # wenn mehr als 5 Spalten dann weiter
                    if (len(row) < 5):
                        continue

                    # Datum in Spalte 1 (row[0])
                    dateStr = row[0]

                    # Zeit in Spalte 2 (row [1])
                    timeStr = row[1]

                    # die Kopfzeilen zusammenfassen
                    if dateStr == "Datum":
                        for s in range(0, len(row)):
                            header.append(row[s] + "-" + prevRow[s])
                        
                        writer_csv.writerow(header)


                        continue
                    
                    
                    try:
                        # das Datum anpassen
                        date1 = datetime.strptime(dateStr, '%Y-%m-%d %H:%M:%S') 
                        row[0] = date1.strftime('%Y-%m-%d') 


                        # die Zeit anpassen
                        hTime = datetime.strptime(timeStr, '%H:%M:%S')
                        row[1] = hTime.strftime('%H:%M')

                    except: 
                        try:
                            # falls zeit so geschrieben ebenfalls anpassen
                            datetime.strptime(dateStr, '%m/%d/%y')
                            row[0] = date.strftime('%Y-%m-%d') 

    
                        except: 
                            # letzte Zeile speichern ( um sie eventuell mit anderer Zeile zu verbinden)
                            prevRow = row
                            continue
                    # alle legitimem Zeilen speichern
                    writer_csv.writerow(row)
                    
                    # all diese Zeilen zählen
                    o += 1
                # Upload auf die Cloud  
                upload_file_cos(cosCreds,localFileEnd,filenameEnd)
            f.close()


ende = time.time()
diff = ende-start

File Downloaded: Halbstd-Werte-Stuttgart-Mitte-SZ_1987.csv
1987
File Uploaded: Halbstd-Werte-Stuttgart-Mitte-SZ_1987_D.csv
File Downloaded: Halbstd-Werte-Stuttgart-Mitte-SZ_1988.csv
1988
File Uploaded: Halbstd-Werte-Stuttgart-Mitte-SZ_1988_D.csv
File Downloaded: Halbstd-Werte-Stuttgart-Mitte-SZ_1989.csv
1989
File Uploaded: Halbstd-Werte-Stuttgart-Mitte-SZ_1989_D.csv
File Downloaded: Halbstd-Werte-Stuttgart-Mitte-SZ_1990.csv
1990
File Uploaded: Halbstd-Werte-Stuttgart-Mitte-SZ_1990_D.csv
File Downloaded: Halbstd-Werte-Stuttgart-Mitte-SZ_1991.csv
1991
File Uploaded: Halbstd-Werte-Stuttgart-Mitte-SZ_1991_D.csv
File Downloaded: Halbstd-Werte-Stuttgart-Mitte-SZ_1992.csv
1992
File Uploaded: Halbstd-Werte-Stuttgart-Mitte-SZ_1992_D.csv
File Downloaded: Halbstd-Werte-Stuttgart-Mitte-SZ_1993.csv
1993
File Uploaded: Halbstd-Werte-Stuttgart-Mitte-SZ_1993_D.csv
File Downloaded: Halbstd-Werte-Stuttgart-Mitte-SZ_1994.csv
1994
File Uploaded: Halbstd-Werte-Stuttgart-Mitte-SZ_1994_D.csv
File Downloaded:

## Informationen

In [27]:
print ("wrote %d (of %d) rows" % (o,i))
print("runtime: %.2fs" %(diff))

wrote 510969 (of 561440) rows
runtime: 42.70s
